### First we have trying the same approch that done in Relince stock 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Input
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
file_path = r"F:\Xai_traderx\experiments\hdfc\hdfcdemo.csv"
df = pd.read_csv(file_path)
df.head()

,date,Close,High,Low,Open,Volume,positive_count,negative_count,neutral_count,avg_sentiment_score,...,SMA_50,EMA_10,EMA_50,EMA_12,EMA_26,RSI_14,MACD,Signal,Volatility,Momentum
0,2020-08-24,1072.344238,1078.008019,1042.200884,1042.536853,18389870,0.0,2.0,1.0,-6.667675e-01,...,1048.585562,1025.204099,1054.382459,1023.134182,1024.227997,48.475442,-1.093815,-9.214172,32.736386,79.294434
1,2020-08-28,1069.896118,1079.975893,1051.176536,1067.448126,19348305,0.0,2.0,0.0,-9.998460e-01,...,1045.239075,1033.329920,1054.990838,1030.328326,1027.610821,49.234122,2.717505,-6.827837,32.122416,78.478271
2,2020-08-31,1071.192139,1102.823505,1056.648439,1084.295871,19250583,0.0,0.0,1.0,-6.124693e-08,...,1042.459937,1040.213960,1055.626183,1036.615066,1030.839067,56.935701,5.776000,-4.307070,35.243832,47.279053
3,2020-09-02,1088.759644,1093.415516,1075.847862,1079.111813,9880841,0.0,1.0,0.0,-9.995025e-01,...,1039.477285,1049.040448,1056.925535,1044.637309,1035.129480,71.688395,9.507829,-1.544090,39.417105,72.142273
4,2020-09-04,1074.504028,1080.359874,1060.200325,1065.576181,11830685,1.0,1.0,0.0,8.826069e-03,...,1036.873823,1053.670190,1057.614887,1049.232189,1038.046113,76.940728,11.186076,1.001943,37.481521,32.303223


In [3]:
selected_features = [
    # Price Action
    'Close', 'High', 'Low', 'Open',
    
    # Short-Term Trends
    'SMA_7', 'SMA_10', 'EMA_10',
    
    # Volatility & Volume
    'bb_upper', 'bb_lower', 'OBV', 'ATR',
    
    # Optional (Test Impact)
    'EMA_50', 'RSI_14', 'MACD_Hist'
]

In [4]:
df_selected = df[selected_features]

In [5]:
# 1. Separate features and target
features = df_selected
target = df[['next_close']]

# 2. Scale features and target separately
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(target)

# 3. Train-test split (no shuffle)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, shuffle=False, test_size=0.2)

# 4. Define lookback
lookback = 4

# 5. Sequence creation function
def create_sequence(X, y, lookback):
    Xs, ys = [], []
    for i in range(len(X) - lookback):
        Xs.append(X[i:(i + lookback)])
        ys.append(y[i + lookback])
    return np.array(Xs), np.array(ys)

# 6. Create sequences
X_train_seq, y_train_seq = create_sequence(X_train, y_train, lookback)
X_test_seq, y_test_seq = create_sequence(X_test, y_test, lookback)

print("Shape of X_train_seq:", X_train_seq.shape)
print("Shape of y_train_seq:", y_train_seq.shape)
print("Shape of X_test_seq:", X_test_seq.shape)
print("Shape of y_test_seq:", y_test_seq.shape)

Shape of X_train_seq: (746, 4, 14)
Shape of y_train_seq: (746, 1)
Shape of X_test_seq: (184, 4, 14)
Shape of y_test_seq: (184, 1)


In [6]:
model = Sequential([
    Input(shape=(lookback, X_train_seq.shape[2])), 
    LSTM(100, return_sequences=True),  # LSTM layer 1
    Dropout(0.2),  # Dropout for regularization
    LSTM(100, return_sequences=False),  # LSTM layer 2
    Dropout(0.2),  # Dropout for regularization
    Dense(1)  # Output layer (predicts Next_Close)
])

In [7]:
optimizer = Nadam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss=Huber(), metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_mae', patience=50, restore_best_weights=True),
    ModelCheckpoint('lstm4dayhdfc.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=25, min_lr=1e-6)  # Learning rate adjustment
]

In [8]:
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=500,
    batch_size=1,
    callbacks=callbacks,
    shuffle=False,  # Keep the order of time series data
    verbose=1
)

print("Training Completed..")

Epoch 1/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.0026 - mae: 0.0575 - val_loss: 0.0084 - val_mae: 0.1165 - learning_rate: 1.0000e-04
Epoch 2/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0027 - mae: 0.0553 - val_loss: 0.0081 - val_mae: 0.1117 - learning_rate: 1.0000e-04
Epoch 3/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0029 - mae: 0.0573 - val_loss: 0.0090 - val_mae: 0.1180 - learning_rate: 1.0000e-04
Epoch 4/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0030 - mae: 0.0568 - val_loss: 0.0140 - val_mae: 0.1531 - learning_rate: 1.0000e-04
Epoch 5/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0027 - mae: 0.0560 - val_loss: 0.0139 - val_mae: 0.1515 - learning_rate: 1.0000e-04
Epoch 6/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0026 - mae: 0.0553 - val_loss: 0.0154 - val_mae: 0.1596 - learning_rate: 1.0000e-04
Epoch 7/500
746/746 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0029 - mae: 0.0558 - val_loss: 0.0166 - val_mae: 0.1670

In [9]:
test_loss, test_mae = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0058 - mae: 0.0927 
Test Loss: 0.008121890015900135
Test MAE: 0.11172332614660263


In [10]:
y_pred_scaled = model.predict(X_test_seq)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step


In [11]:
# Reshape if needed
y_pred_scaled = y_pred_scaled.reshape(-1, 1)
y_test_seq = y_test_seq.reshape(-1, 1)

# Inverse transform
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test_seq)

In [12]:
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)
percentage_error = (mae / np.mean(y_true)) * 100
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2 Score: {r2:.4f}")
print(f"Percentage Error: {percentage_error:.2f}%")

MAE: 108.1189
RMSE: 123.3393
R2 Score: -0.8429
Percentage Error: 6.33%


The RMSE and MAE are higher than expected, which means  model struggles to accurately predict the next day's price for HDFC stock.